# Quantum Channels Simulations

### Import necessary libraries

In [1]:
import numpy as np
import plotly.graph_objects as go

### Function to create an interactive 3D plot for visualizing transformations

In [21]:
def interactivePlot(original_vectors, transformed_vectors, legend, title, lines=True):
    # Split into x, y, z components
    orig_x, orig_y, orig_z = original_vectors[:, 0], original_vectors[:, 1], original_vectors[:, 2]
    trans_x, trans_y, trans_z = transformed_vectors[:, 0], transformed_vectors[:, 1], transformed_vectors[:, 2]
    
    # Create a Plotly figure
    fig = go.Figure()

    # Add original vectors to the plot (blue color)
    fig.add_trace(go.Scatter3d(
        x=orig_x, y=orig_y, z=orig_z,
        mode='markers',
        marker=dict(size=2.5, color='blue', opacity=0.7),
        name=legend[0]
    ))

    # Add transformed vectors to the plot (red color)
    fig.add_trace(go.Scatter3d(
        x=trans_x, y=trans_y, z=trans_z,
        mode='markers',
        marker=dict(size=2.5, color='red', opacity=0.7),
        name=legend[1]
    ))

    if lines:
        # Add lines connecting original and transformed points
        for i in range(len(orig_x)):
            fig.add_trace(go.Scatter3d(
                x=[orig_x[i], trans_x[i]],  # X coordinates of the line
                y=[orig_y[i], trans_y[i]],  # Y coordinates of the line
                z=[orig_z[i], trans_z[i]],  # Z coordinates of the line
                mode='lines',
                line=dict(color='grey', width=2),  # Thin grey lines
                showlegend=False  # Don't show these lines in the legend
            ))

    # Add a unit sphere
    u = np.linspace(0, 2 * np.pi, 30)  # 30 points around the sphere
    v = np.linspace(0, np.pi, 30)  # 30 points from pole to pole
    u, v = np.meshgrid(u, v)

    x = np.cos(u) * np.sin(v)  # Sphere x-coordinates
    y = np.sin(u) * np.sin(v)  # Sphere y-coordinates
    z = np.cos(v)  # Sphere z-coordinates

    fig.add_trace(go.Surface(
        x=x, y=y, z=z,
        opacity=0.3,  # Make it semi-transparent
        colorscale='blues',  # Neutral color
        showscale=False  # Hide color scale
    ))

    # Update layout for titles and axes labels
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        ),
        legend=dict(x=0, y=1),
        # Set the size of the plot
        width=1000,  # Increase width
        height=800   # Increase height
    )

    # Show the interactive plot
    fig.show()

In [17]:
# Single-Qubit Gates
I = np.eye(2, dtype=complex)  # Identity

X = np.array([[0, 1],
              [1, 0]], dtype=complex)  # Pauli-X

Y = np.array([[0, -1j],
              [1j, 0]], dtype=complex)  # Pauli-Y

Z = np.array([[1, 0],
              [0, -1]], dtype=complex)  # Pauli-Z

H = (1/np.sqrt(2)) * np.array([[1, 1],
                               [1, -1]], dtype=complex)  # Hadamard

S = np.array([[1, 0],
              [0, 1j]], dtype=complex)  # Phase (S)

T = np.array([[1, 0],
              [0, np.exp(1j * np.pi / 4)]], dtype=complex)  # T gate

# Rotation Gates
def RX(theta):
  return np.array([[np.cos(theta/2), -1j * np.sin(theta/2)],
                     [-1j * np.sin(theta/2), np.cos(theta/2)]], dtype=complex)

def RY(theta):
  return np.array([[np.cos(theta/2), -np.sin(theta/2)],
                     [np.sin(theta/2), np.cos(theta/2)]], dtype=complex)

def RZ(theta):
  return np.array([[np.exp(-1j * theta / 2), 0],
                     [0, np.exp(1j * theta / 2)]], dtype=complex)

### Function to generate Quantum States

In [4]:
# if output_bloch_vector = true this function outputs bloch vector otherwise outputs density matrix
def getDensityMatrix(state, output_bloch_vector):
    # if state = "pure" this function picks up states from bloch sphere
    if state == "pure":
        randomVector = np.random.uniform(-1, 1, 3)
        bloch_vector = randomVector / np.linalg.norm(randomVector)

    # if state = "mixed" this function picks up states from bloch ball
    elif state == "mixed":
        while True:
            bloch_vector = np.random.uniform(-1, 1, 3)
            if np.linalg.norm(bloch_vector) <= 1:
                break

    rx, ry, rz = bloch_vector
    output = 0.5 * (I + rx * X + ry * Y + rz * Z)

    if output_bloch_vector:
        output = bloch_vector
    return output

### Kraus Operators for Channels

In [5]:
def depolarizingKrausOperators(p):
  K0 = np.sqrt(1 - p) * I
  K1 = np.sqrt(p / 3) * X
  K2 = np.sqrt(p / 3) * Y
  K3 = np.sqrt(p / 3) * Z
  return [K0, K1, K2, K3]

def amplitudeDampingKrausOperators(p):
  K0 = np.array([[1, 0], [0, np.sqrt(1 - p)]])
  K1 = np.array([[0, np.sqrt(p)], [0, 0]])
  return [K0, K1]

def generalPhaseDampingKrausOperators(p):
  K0 = np.sqrt(1 - p) * np.eye(2)
  K1 = np.sqrt(p) * np.array([[1, 0], [0, 0]])
  K2 = np.sqrt(p) * np.array([[0, 0], [0, 1]])
  return [K0, K1, K2]

def phaseDampingKrausOperators(p):
  K0 = np.sqrt(p) * np.eye(2)
  K1 = np.sqrt(1 - p) * Z
  return [K0, K1]

def bitFlipKrausOperators(p):
  K0 = np.sqrt(1 - p) * np.eye(2)
  K1 = np.sqrt(p) * X
  return [K0, K1]

def phaseFlipKrausOperators(p):
  K0 = np.sqrt(1 - p) * np.eye(2)
  K1 = np.sqrt(p) * Y
  return [K0, K1]

def bitPhaseFlipKrausOperators(p):
  K0 = np.sqrt(1 - p) * np.eye(2)
  K1 = np.sqrt(p) * Y
  return [K0, K1]

### Generate Original Quantum States and Their Transformed States

In [9]:
def computeBlochVector(rho):
    # Compute the Bloch vector representation of a given quantum state (density matrix)
    rx = np.trace(np.dot(rho, X)).real  # Projection onto X-axis
    ry = np.trace(np.dot(rho, Y)).real  # Projection onto Y-axis
    rz = np.trace(np.dot(rho, Z)).real  # Projection onto Z-axis

    return np.array([rx, ry, rz])
    
def apply_quantum_channel(rho, kraus_operators):
    # Apply a quantum channel to a given density matrix using a set of Kraus operators
    transformed_rho = np.zeros_like(rho, dtype=complex)
    for K in kraus_operators:
        transformed_rho += K @ rho @ K.conj().T  # Apply Kraus operator
    
    return transformed_rho

def generateData(state, num_samples, transformation):
    rhosVect = []  # List to store Bloch vectors of input states
    sigmasVect = []  # List to store Bloch vectors of transformed states
    
    for i in range(num_samples):
        blochVector = getDensityMatrix(state, output_bloch_vector=True)  # Get Bloch vector
        rhosVect.append(blochVector)
        
        # Reconstruct the density matrix from the Bloch vector
        rx, ry, rz = blochVector
        rho = 0.5 * (I + rx * X + ry * Y + rz * Z)  
        
        # Apply the quantum transformation
        sigma = transformation(rho)
        
        # Compute the Bloch vector of the transformed state
        sigmaVect = computeBlochVector(sigma)
        sigmasVect.append(sigmaVect)

    return np.array(rhosVect), np.array(sigmasVect)

In [25]:
num_samples = 500
p = 0.5
transformation = lambda rho: apply_quantum_channel(rho, depolarizingKrausOperators(p))
rhosVect, sigmasVect = generateData("pure", num_samples, transformation)
interactivePlot(rhosVect, sigmasVect, ["Original States", "Transformed States"], f"Representation of Depolarizing Channel with probability = {p}", False)

In [29]:
num_samples = 500
p = 0.5
transformation = lambda rho: apply_quantum_channel(rho, amplitudeDampingKrausOperators(p))
rhosVect, sigmasVect = generateData("pure", num_samples, transformation)
interactivePlot(rhosVect, sigmasVect, ["Original States", "Transformed States"], f"Representation of Amplitude Damping Channel with probability = {p}", False)

In [30]:
num_samples = 500
p = 0.5
transformation = lambda rho: apply_quantum_channel(rho, generalPhaseDampingKrausOperators(p))
rhosVect, sigmasVect = generateData("pure", num_samples, transformation)
interactivePlot(rhosVect, sigmasVect, ["Original States", "Transformed States"], f"General Phase Damping Channel with probability = {p}", False)

In [31]:
num_samples = 500
p = 0.25
transformation = lambda rho: apply_quantum_channel(rho, phaseDampingKrausOperators(p))
rhosVect, sigmasVect = generateData("pure", num_samples, transformation)
interactivePlot(rhosVect, sigmasVect, ["Original States", "Transformed States"], f"Phase Damping Channel with probability = {p}", False)

In [32]:
num_samples = 500
p = 0.25
transformation = lambda rho: apply_quantum_channel(rho, bitFlipKrausOperators(p))
rhosVect, sigmasVect = generateData("pure", num_samples, transformation)
interactivePlot(rhosVect, sigmasVect, ["Original States", "Transformed States"], f"Bit Flip Channel with probability = {p}", False)

In [33]:
num_samples = 500
p = 0.25
transformation = lambda rho: apply_quantum_channel(rho, phaseFlipKrausOperators(p))
rhosVect, sigmasVect = generateData("pure", num_samples, transformation)
interactivePlot(rhosVect, sigmasVect, ["Original States", "Transformed States"], f"Phase Flip Channel with probability = {p}", False)

In [34]:
num_samples = 500
p = 0.25
transformation = lambda rho: apply_quantum_channel(rho, bitPhaseFlipKrausOperators(p))
rhosVect, sigmasVect = generateData("pure", num_samples, transformation)
interactivePlot(rhosVect, sigmasVect, ["Original States", "Transformed States"], f"Bit Phase Flip Channel with probability = {p}", False)